<a href="https://colab.research.google.com/github/Mai-Fakhry/Optical-character-recognition-OCR-/blob/main/NoteBooks/OCR_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Mai-Fakhry/Optical-character-recognition-OCR-

Cloning into 'Optical-character-recognition-OCR-'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 45 (delta 12), reused 15 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 91.75 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [3]:
!pip install kaggle

In [17]:
!ls /content

kaggle.json  Optical-character-recognition-OCR-  sample_data  template


In [18]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [19]:
!kaggle datasets download -d harieh/ocr-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/harieh/ocr-dataset
License(s): CC0-1.0
100% 153M/153M [00:00<00:00, 1.60GB/s]
100% 153M/153M [00:00<00:00, 1.60GB/s]


In [1]:
import os
import zipfile
import shutil
from PIL import Image
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [20]:
zip_path = "/content/Optical-character-recognition-OCR-/Datasets/template.zip"
extract_path = "/content"


os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

print("Extracted to:", extract_path)
print("Contents:", os.listdir(extract_path))

Extracted to: /content
Contents: ['.config', 'dataset', 'template', 'Optical-character-recognition-OCR-', 'kaggle.json', 'sample_data']


In [58]:
template_dir = "/content/template/uppercase"
dataset_dir = "/content/dataset"
import string

uppercase_letters = list(string.ascii_uppercase)
lowercase_letters = list(string.ascii_lowercase)
digits = [str(i) for i in range(10)]


In [60]:
def preprocess_image(img, size=(40, 60)):
    if img is None:
        return None

    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_bin = cv2.adaptiveThreshold(
        img,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        15,
        3
    )

    coords = cv2.findNonZero(img_bin)
    if coords is None:
        return None

    x, y, w, h = cv2.boundingRect(coords)
    cropped = img_bin[y:y+h, x:x+w]
    if cropped.size == 0:
        return None


    h_c, w_c = cropped.shape
    max_side = max(h_c, w_c)
    pad_vert = (max_side - h_c) // 2
    pad_horz = (max_side - w_c) // 2
    cropped_padded = cv2.copyMakeBorder(
        cropped,
        pad_vert,
        max_side - h_c - pad_vert,
        pad_horz,
        max_side - w_c - pad_horz,
        cv2.BORDER_CONSTANT,
        value=0
    )

    resized = cv2.resize(cropped_padded, size)
    return resized / 255.0



In [51]:
def extract_hog(img):
    features = hog(
        img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    return features


In [61]:
X = []
y = []

dataset_dir = "/content/dataset"

# --- Uppercase ---
for char in uppercase_letters:
    folder = os.path.join(dataset_dir, f"{char}_U")
    if not os.path.exists(folder):
        continue
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        processed = preprocess_image(img)
        if processed is None:
            continue
        X.append(extract_hog(processed))
        y.append(char)

# --- Lowercase ---
for char in lowercase_letters:
    folder = os.path.join(dataset_dir, f"{char}_L")
    if not os.path.exists(folder):
        continue
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        processed = preprocess_image(img)
        if processed is None:
            continue
        X.append(extract_hog(processed))
        y.append(char)

# --- Digits ---
for digit in digits:
    folder = os.path.join(dataset_dir, digit)
    if not os.path.exists(folder):
        continue
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        processed = preprocess_image(img)
        if processed is None:
            continue
        X.append(extract_hog(processed))
        y.append(digit)

X = np.array(X)
y = np.array(y)

print("Total samples:", len(X))
print("Total classes:", len(np.unique(y)))
print("Classes:", np.unique(y))


Total samples: 209679
Total classes: 62
Classes: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r'
 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [63]:
svm_model = LinearSVC(C=1.0, max_iter=5000)
svm_model.fit(X_train, y_train)


LinearSVC(max_iter=5000)

In [64]:
y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.866010110644792
              precision    recall  f1-score   support

           0       0.68      0.67      0.67       693
           1       0.90      0.86      0.88       693
           2       0.95      0.94      0.95       693
           3       0.86      0.94      0.90       693
           4       0.97      0.94      0.95       693
           5       0.97      0.92      0.95       693
           6       0.97      0.94      0.95       693
           7       0.93      0.94      0.93       693
           8       0.95      0.93      0.94       693
           9       0.91      0.94      0.93       693
           A       0.90      0.89      0.89       691
           B       0.93      0.93      0.93       691
           C       0.84      0.77      0.80       691
           D       0.88      0.91      0.90       691
           E       0.92      0.88      0.90       691
           F       0.90      0.90      0.90       691
           G       0.88      0.90      0.89       691